In [ ]:
import numpy as np
import pandas as pd
from pyens.models import Flywheel, OCV, EcmCell
from pyens.utilities import ivp
from pyens.simulations import Data, Current, Learner,Simulator
import os
import matplotlib.pyplot as plt

In [ ]:
ocv = [3.3, 3.5, 3.55, 3.6, 3.65, 3.68, 3.70, 3.8, 3.95, 4.0, 4.1]
soc = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

c1 = OCV(name="curve1", ocv=ocv, soc=soc)

c1.display()

In [ ]:
param = {
 "R0": 0.034,
 "R1": 0.022,
 "C1": 1500,
 "R2": 0.019,
 "C2": 65000,
 "CAP": 15,
 "ce": 0.96,
 "v_limits": [2.5, 4.5],
 "SOC_RANGE": [0.0, 100.0],
}

m1 = EcmCell(name="cell_model1", parameters=param, curve=c1)
# m1.display()

In [ ]:
dt = 0.1
CURR_EXCITATION = 7.5
HOUR = 3600.0
h_steps = [1.75, 0.25, 0.25, 0.25, 1, 1, 1]
current_steps = [
 -CURR_EXCITATION,
 -CURR_EXCITATION / 2,
 -CURR_EXCITATION / 4,
 -CURR_EXCITATION / 8,
 0.0,
 CURR_EXCITATION,
 0.0,
]

initial_soc = 0.0
t_steps = [value * HOUR for value in h_steps]
total_time = sum(t_steps)
samples = int(total_time / dt)

time_np = np.linspace(0.0, total_time, samples)
step_cur = Current(name="current1")

In [ ]:
for t_step, current_step in zip(t_steps, current_steps):
 step_cur.add_step(current_step, int(t_step / dt))

data = {"time": time_np, "current": step_cur.current}
df = pd.DataFrame(data)
d1 = Data(name="current_excite", df=df)

In [ ]:
s1 = Simulator(name="simulator1")
s1.attach(m1).attach(d1)

In [ ]:
sol = s1.run(
    pair=("cell_model1", "current_excite"),
    x0=np.array([0, 0, initial_soc]),
    config={"solver_type": "adaptive", "solution_name": "sol1"},
)

sol.disp(["current", "soc", "vt"])

In [ ]:
TESTDATA_FILEPATH = os.path.join(os.path.dirname(os.getcwd()),'tests/CS2_3_9_28_11.csv')
schema = {
    "Test_Time(s)": "time",
    "Current(A)": "current",
    "Voltage(V)": "vt",
    "rsv_i_dir": True,
}
d1 = Data(name='d1', df=None)
d1.fetch_file(TESTDATA_FILEPATH, schema=schema)
# save the calculation time
# d1.df=d1.df[d1.df['time']>3000].reset_index()

In [ ]:

# l1=Learner(name="l1")
# # initial guess for R0, R1,C1,R2,C2
# p0=[0.1,
# 0.005,
# 1500,
# 0.04,
# 65000]
# x0=np.array([0, 0, 100])
# config={"solver_type": None, "solution_name": "sol1"}
# res=l1.fit_parameters(p0,d1,c1,config,x0,"ls")
# R0,R1,C1,R2,C2=res.x
# print("R0:",R0,"R1:",res.x[1],"C1:",res.x[2],"R2:",res.x[3],"C2:",res.x[4])


In [ ]:
param_sim = {
 "R0": 0.034,
 "R1": 0.022,
 "C1": 1500,
 "R2": 0.019,
 "C2": 65000,
 "CAP": d1.df['Discharge_Capacity(Ah)'].max(),
 "ce": 0.96,
 "v_limits": [2.5, 4.5],
 "SOC_RANGE": [0.0, 100.0],
}
m_sim = EcmCell(name="cell_model_sim", parameters=param_sim, curve=c1)
s2=Simulator(name="s2")
s2.attach(d1).attach(m_sim)
l2=Learner(name="l2")
l2.attach(s2)
m_d_s=('cell_model_sim','d1','s2')
x0_sim=np.array([0, 0, 100])
config={"solver_type": None, "solution_name": "sol1"}
method="global"
res=l2.fit_parameters(m_d_s,config,x0_sim,method)

In [ ]:
# d1.df['Charge_Capacity(Ah)'].max()
# d1.df['Discharge_Capacity(Ah)'].max()
res.x

In [ ]:
d2=s2.get("sol1")
d1.df.vt.min()
# d2.df.current.iloc[11104]

In [ ]:
# res=l1.run_sim(p0,d1.get_current,d1.df.time,c1,config,x0)
# len(res[0])
# plt.plot(d1.df.vt,'.',label="true")
# plt.plot(res[0],'.',label="pred")
# plt.legend()

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
d2=s2.get("sol1")
y_true=d1.df.vt
y_pred=d2.df.vt

rmse=mean_squared_error(y_true, y_pred, squared=False)
r_square=r2_score(y_true, y_pred)
print('rmse:',rmse,"r square:",r_square)

plt.plot(y_true,'.',label="true")
plt.plot(y_pred,'.',label="pred")

plt.xlabel('time (ms)')
plt.ylabel('vt')
plt.legend()
plt.show()

In [ ]:
plt.plot(d1.df.current.iloc[300:3740],'.',label="curr")
d2.df.time.iloc[3679]-d2.df.time.iloc[3678]
d1.df[['time','current']].iloc[3678:3680]
# d1.df.time

In [ ]:
d1.df.time.iloc[100:4000].diff().idxmax()
d1.df.time.iloc[3700:4000].diff()
# d2.df[['time','current','vt']].iloc[3650:3710]

In [ ]:
res